In [15]:
def __indent(elem, level=0):
    i = "\n" + level*"\t"
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "\t"
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            __indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

In [16]:
from feature_selection.workdir.feature_selection import feature_selection_regression
import pandas as pd
raw_data = pd.read_csv('./feature_selection/workdir/1561616415645612165/test.csv')
y = raw_data['7']
X = raw_data.iloc[:, 1-1: 6]
feature_name = list(X.columns)
feature_selection_result = feature_selection_regression(X,y,feature_name)

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


In [17]:
feature_selection_result

,Feature,ElasticNet,Lasso,LightGBM,Pearson,RF,SVR,SVR-RFE,Variance,XGBoost,Total
1,5,6.451613,6.451613,NaN,6.451613,22.580645,9.677419,19.354839,3.225806,25.806452,100.000000
2,6,6.451613,6.451613,NaN,6.451613,0.000000,9.677419,19.354839,3.225806,25.806452,77.419355
3,4,6.451613,6.451613,NaN,6.451613,0.000000,9.677419,19.354839,3.225806,25.806452,77.419355
4,3,6.451613,6.451613,NaN,6.451613,0.000000,9.677419,19.354839,3.225806,25.806452,77.419355
5,1,6.451613,6.451613,NaN,6.451613,0.000000,0.000000,19.354839,3.225806,25.806452,67.741935
6,2,6.451613,0.000000,NaN,6.451613,0.000000,0.000000,19.354839,3.225806,25.806452,61.290323


In [29]:
def _add_info_xml(picture_names, feature_selection_result) -> None:
    try:
        import xml.etree.cElementTree as ET
    except ImportError:
        import xml.etree.ElementTree as ET
    import os
    # Back up result.xml
    os.system('cp ./result.xml ./result_before.xml')  # Linux
    # os.system('copy .\\result.xml .\\result_before.xml')    # Win

    tree = ET.parse("./result.xml")
    root = tree.getroot()
    output = tree.find('output')
    SHAP = output.find('SHAP')

    element_picture = ET.Element('picture')
    for picture_name in picture_names:
        element_file, element_name, element_url = ET.Element('file'), ET.Element('name'), ET.Element('url')
        element_name.text, element_url.text = picture_name, picture_name
        element_picture.append(element_file)
        element_file.append(element_name)
        element_file.append(element_url)
    SHAP.append(element_picture)
    
    feature_selection = output.find('feature-selection')
    element_catalog = ET.Element('catalog')
    for _, data in feature_selection_result.iterrows():
        feature = ET.Element('feature')
        for key, value in zip(data.index, data):
            method = ET.Element(key)
            method.text = str(value)
            feature.append(method)
        element_catalog.append(feature)
    feature_selection.append(element_catalog)
    # Save
    __indent(root)
    ET.tostring(root, method='xml')

    tree.write('result.xml', encoding='utf-8', xml_declaration=True)
    with open('result.xml', 'r') as fp:
        lines = [line for line in fp]
        lines.insert(1, '<?xml-stylesheet type="text/xsl" href="/XSLTransform/feature_selection.xsl" ?>\n')
    with open('result.xml', 'w') as fp:
        fp.write(''.join(lines))
        


In [30]:
_add_info_xml(['1', '2'], feature_selection_result)

In [59]:
import json
with open('parameters.json','r',encoding='utf8')as fp:
    parameters = json.load(fp)
parameters['feature_start'] = int(parameters['feature_start'])
parameters['feature_end'] = int(parameters['feature_end'])
parameters

{'inputCSV': 'a.csv',
 'feature_start': 0,
 'feature_end': '8',
 'target': 'target',
 'single': '1,2,3',
 'interactionA': '0,1,2',
 'interactionB': '3,4,5'}